In [95]:
import pandas as pd
import numpy as np
import os

In [96]:
main_datasets = ["BACE_class", "BACE_regre", "ROR_class", "ROR_regre"]
top5 = {}

best_models = {}

In [97]:
for dataset in main_datasets:
    df = pd.read_csv(r"Merged runs/" + dataset + ".csv")
    df = df.drop(["Unnamed: 0"], axis=1)
    #print(df.columns)

    regre = False
    score_column = "roc_auc"
    if dataset[-5:] == "regre": 
        regre = True
        score_column = "mse"


    df.sort_values(score_column, ascending=regre, inplace=True)

    top5[dataset] = df.head(5)

    best = {}

    for index, row in df.iterrows():
        if len(best) >= 7:
            break
        if not row['model'] in best.keys():
            best[row['model']] = [row[score_column], row['hyperparams']]

    best_models[dataset] = best

### Top 5 scores on a dataset

In [98]:
top5[main_datasets[0]]

,model,hyperparams,roc_auc,train_roc_auc,accuracy,train_accuracy,precision,train_precision,recall,train_recall,f1,train_f1,split,preprocessing
0,SVC,"{'C': 0.01, 'kernel': 'poly', 'gamma': 0.05}",0.880874,0.873432,0.881579,0.872153,0.887324,0.832268,0.863014,0.883051,0.875000,0.856908,0.9,Descriptors
1,MLPClassifier,"{'hidden_layer_sizes': (50, 50), 'activation':...",0.876626,0.950669,0.875000,0.946363,0.837500,0.902022,0.917808,0.983051,0.875817,0.940795,0.9,DescFing
2,LogisticRegression,"{'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}",0.874025,0.920109,0.875000,0.919177,0.885714,0.890879,0.849315,0.927119,0.867133,0.908638,0.9,DescFing
3,MLPClassifier,"{'hidden_layer_sizes': (100,), 'activation': '...",0.869256,0.889231,0.868421,0.884644,0.844156,0.829528,0.890411,0.923729,0.866667,0.874098,0.9,Descriptors
4,MLPClassifier,"{'hidden_layer_sizes': (100,), 'activation': '...",0.867696,0.961201,0.868421,0.960323,0.873239,0.942244,0.849315,0.967797,0.861111,0.954849,0.9,Fingerprints


### Best performance of each model on the dataset

In [101]:
best_models[main_datasets[0]]

{'SVC': [0.8808739379226634, "{'C': 0.01, 'kernel': 'poly', 'gamma': 0.05}"], 'MLPClassifier': [0.8766256285763829, "{'hidden_layer_sizes': (50, 50), 'activation': 'relu', 'alpha': 0.01, 'max_iter': 500, 'solver': 'sgd'}"], 'LogisticRegression': [0.8740246228541703, "{'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}"], 'GradientBoostingClassifier': [0.8671753077856772, "{'max_depth': 3, 'loss': 'exponential', 'n_estimators': 200, 'learning_rate': 1.0, 'min_impurity_decrease': 0.05}"], 'RandomForestClassifier': [0.8613663950060689, "{'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 2, 'min_impurity_decrease': 0, 'n_estimators': 200}"], 'XGBClassifier': [0.8555574822264608, "{'max_depth': 7, 'eta': 0.1, 'gamma': 0}"], 'DecisionTreeClassifier': [0.8374097834803528, "{'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 1, 'min_impurity_decrease': 0, 'criterion': 'log_loss', 'splitter': 'best'}"]}
{'SVR': [0.3864183150131471, "{'C': 1, 'kernel': 'rbf', 'epsilon': 0.01, 

In [ ]:
best_models[main_datasets[1]]

In [ ]:
best_models[main_datasets[2]]

In [ ]:
best_models[main_datasets[3]]

### 